## Import MNIST Images - Deep Learning with PyTorch 14

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

Dataset link: https://www.tensorflow.org/datasets/catalog/mnist

In [5]:
# Convert MNIST image files into Tensor of 4-Dimentions (# of images, Height, Width, Color Channel)
transform = transforms.ToTensor()